# Create `MBTiles`
In this notebook we will transforms vector type data into MBTiles using tippecanoe.

# Setup
## Library import
We import all the required Python libraries

In [504]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess
from dotenv import load_dotenv
import glob
from google.cloud import storage
from google.oauth2 import service_account
import ipyleaflet as ipyl

## Utils

**read_indicators**

In [399]:
def read_indicators(dataset, indicator, gdf_vectors, geo_type='Municipios'):

    gdf_ids = {'Provincias': ['CO_CCAA', 'CO_PROVINC'],
    'Comunidades autonomas': ['CO_CCAA'],
    'Municipios': ['CODIGOINE'],
    'Rejilla': ['grid_id']}

    gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
    'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
    'Municipios': ['NAMEUNIT', 'CODIGOINE'],
    'Rejilla': ['grid_id']}

    geo_name = geo_type.lower().replace(' ', '_')

    df = pd.read_csv(f'../../datasets/processed/{dataset}/{dataset}_{geo_name}.csv')
    df = df.astype(dict(zip(gdf_ids[geo_type], [int]*len(gdf_ids[geo_type]))))
    gdf = df[(df['indicator'] == indicator)]
    gdf = gpd.GeoDataFrame(pd.merge(gdf, gdf_vectors[geo_type][gdf_columns[geo_type]+['geometry']].astype(dict(zip(gdf_ids[geo_type], [int]*len(gdf_ids[geo_type])))),\
         how='left', on=gdf_ids[geo_type]))

    gdf = gdf[gdf_columns[geo_type] + ['dataset', 'indicator', 'scenario', 'value', 'year', 'unit', 'geometry']]

    return gdf

**reorganize_data**

In [505]:
def reorganize_data(gdf, geo_type='Municipios'):
    gdf_id = {'Provincias': 'CO_PROVINC',
    'Comunidades autonomas': 'CO_CCAA',
    'Municipios': 'CODIGOINE',
    'Rejilla': 'grid_id'}

    gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
    'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
    'Municipios': ['NAMEUNIT', 'CODIGOINE'],
    'Rejilla': ['grid_id']}

    gdf_new = gpd.GeoDataFrame(columns=gdf_columns[geo_type]+['dataset', 'indicator', 'unit', 'values', 'geometry'])
    for code_id in gdf[gdf_id[geo_type]].unique():
        #LD = gdf[gdf[gdf_id[geo_type]] == code_id][['scenario', 'year', 'value']].to_dict('records')
        LD = json.loads(gdf[gdf[gdf_id[geo_type]] == code_id][['scenario', 'year', 'value']].to_json(orient='records'))
        if geo_type == 'Rejilla':
            v = {k: {dic['year']: dic['value'] for dic in LD if dic['scenario'] == k} for k in ['baseline', 'rcp45', 'rcp85']}
        else:
            v = {k: {dic['year']: dic['value'] for dic in LD if dic['scenario'] == k} for k in ['rcp45', 'rcp85']}

        gdf_tmp = gdf[gdf[gdf_id[geo_type]] == code_id][gdf_columns[geo_type]].iloc[:1]
        gdf_tmp['dataset'] = gdf['dataset'].iloc[0]
        gdf_tmp['indicator'] = gdf['indicator'].iloc[0]
        gdf_tmp['unit'] = gdf['unit'].iloc[0]
        gdf_tmp['values'] = [v]
        gdf_tmp['geometry'] = gdf[gdf[gdf_id[geo_type]] == code_id]['geometry'].iloc[:1]

        gdf_new = pd.concat([gdf_new, gdf_tmp])

    return gdf_new

**create_mbtiles**

In [358]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**mbtile_to_pbf**

In [359]:
def mbtile_to_pbf(input_mbtiles, output_path):
    """
    Use mb-util to convert mbtiles to to pbf in z/x/y.pbf form.
    To export MBTiles to disk, specify a directory that does not yet exist.
    """
    cmd = f"mb-util {input_mbtiles} {output_path} --image_format=pbf"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**upload_local_directory_to_gcs**

In [360]:
def upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path):
    """Uploads a directory to the bucket."""
    
    credentials_dict = {
          "type": "service_account",
          "project_id": os.getenv("PROJECT_ID"),
         "private_key_id": os.getenv("PRIVATE_KEY_ID"),
          "private_key":os.getenv("PRIVATE_KEY"),
         "client_email": os.getenv("CLIENT_EMAIL"),
          "token_uri":os.getenv("TOKEN_URI"),   
    } 
    

    credentials = service_account.Credentials.from_service_account_info(credentials_dict)
    storage_client = storage.Client(project='project_id', credentials=credentials)
    
    bucket = storage_client.bucket(bucket_name)
    rel_paths = glob.glob(local_path + '/**', recursive=True)

    for local_file in rel_paths:
        remote_path = f'{destination_blob_path}{"/".join(local_file.split(os.sep)[len(local_path.split(os.sep))-1:])}'
        if os.path.isfile(local_file):
            blob = bucket.blob(remote_path)
            print(
                "File {} uploaded to {}.".format(
                    local_file, remote_path
                )
            )
            blob.upload_from_filename(local_file)

**vector_tile_processing**

In [513]:
def vector_tile_processing(path, dataset, folder_name, gdf, geo_type='Municipios', pbf_format=True, upload_to_GCS=True):
    geo_name = geo_type.lower().replace(' ', '_')
    
    # Save GeoDataFrame as `GeoJSON`
    #gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")
    with open(f"{path}/{folder_name}/{geo_name}.json", 'w') as outfile:
        outfile.write(gdf.to_json())

    # Create MBTiles
    print('Creating MBTiles')
    source_path = f"{path}/{folder_name}/{geo_name}.json"
    mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
    #create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
    create_mbtiles(source_path, mbtil_path, folder_name, opts="-Z1 -z10 --drop-densest-as-needed --force")

    # Convert from `MBTiles` to `pbf` in {z}/{x}/{y}.pbf format
    print('Converting from MBTiles to pbf')
    if pbf_format:
        output_path = f"{path}/{folder_name}/{geo_name}"
        mbtile_to_pbf(mbtil_path, output_path)

    # Upload `pbf` files to GCS
    print('Uploading pbf files to GCS')
    if upload_to_GCS:
        load_dotenv()

        bucket_name = 'ecf-agricultural-climate-impact'
        local_path = f"{path}/{folder_name}/{geo_name}"
        destination_blob_path = f"MBTiles/{folder_name}/{geo_name}/"

        upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**all_processing**

In [362]:
def all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types = ['Provincias', 'Municipios']):
    for geo_type in geo_types:
        print(geo_type)
        # Read data
        gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

        # Reorganice data
        gdf = reorganize_data(gdf, geo_type) 

        # Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format
        vector_tile_processing(path, dataset, folder_name, gdf, geo_type, pbf_format=True, upload_to_GCS=True)

# Data import

## Comunidades autonomas

In [363]:
comunidades = gpd.read_file(f'../../datasets/processed/comunidades.geojson')
comunidades.sort_values(['CO_CCAA'], inplace = True)

## Provincias

In [364]:
provincias = gpd.read_file(f'../../datasets/raw/georegions/Provincias/Provincias.shp')
provincias.sort_values(['CO_CCAA', 'CO_PROVINC'], inplace = True)

## Municipios de España

In [365]:
municipios = gpd.read_file(f'../../datasets/raw/georegions/Municipios/Municipios_IGN.shp')
municipios.sort_values(['CODNUT1', 'CODNUT2', 'CODNUT3', 'CODIGOINE'], inplace = True)

# Remove Canarias, Ceuta, and Melilla
municipios = municipios[~municipios['CODNUT2'].isin(['ES70', 'ES63', 'ES64'])]
municipios = municipios.reset_index(drop=True)

## Rejilla viñedos

In [520]:
rejilla = gpd.read_file('../../datasets/processed/rejilla_viñedos.geojson')
rejilla.sort_values(['grid_id'], inplace = True)

**Put all vector data together** 

In [521]:
gdf_vectors = {'Comunidades autonomas': comunidades,
"Provincias": provincias,
"Municipios": municipios,
"Rejilla": rejilla}

# Create tiles

## Aumento de temperaturas proyectadas por municipio
**Create all tiles**

In [153]:
dataset = 'climate_indicators'
indicator = 'mean_Tmean_Yearly_change'
path = '../../datasets/processed/MBTiles'
folder_name = 'Aumento_temperaturas'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Create `MBTile` for a single `scenario` and `year`**

In [367]:
dataset = 'climate_indicators'
indicator = 'mean_Tmean_Yearly_change'
path = '../../datasets/processed/MBTiles'
folder_name = 'Aumento_temperaturas'
geo_type = 'Municipios'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)
gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2035 - 2065')]
gdf.head()

,NAMEUNIT,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
6,Abegondo,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,1.182954,2035 - 2065,ºC,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
16,Ames,15002,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,1.212545,2035 - 2065,ºC,"POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
26,Aranga,15003,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,1.212774,2035 - 2065,ºC,"POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
36,Ares,15004,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,1.119780,2035 - 2065,ºC,"MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
46,Arteixo,15005,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,1.094536,2035 - 2065,ºC,"MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


In [340]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z12 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Aumento_temperaturas/municipios.mbtiles -l Aumento_temperaturas -Z1 -z12 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Aumento_temperaturas/municipios.json


8115 features, 10702482 bytes of geometry, 52404 bytes of separate metadata, 259145 bytes of string pool


Task created
Finished processing


  99.9%  12/2037/1540  


## Duración de las sequías
**Create all tiles**

In [161]:
dataset = 'climate_indicators'
indicator = 'dry-spells_maximum-length'
path = '../../datasets/processed/MBTiles'
folder_name = 'Duracion_sequias'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Create `MBTile` for a single `scenario` and `year`**

In [368]:
dataset = 'climate_indicators'
indicator = 'dry-spells_maximum-length'
path = '../../datasets/processed/MBTiles'
folder_name = 'Duracion_sequias'
geo_type = 'Municipios'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2041 - 2060')]
gdf.head()

,NAMEUNIT,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
4,Abegondo,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.331772,2041 - 2060,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
11,Ames,15002,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.904419,2041 - 2060,days,"POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
18,Aranga,15003,Bioclimatic indicators,dry-spells_maximum-length,rcp45,33.225460,2041 - 2060,days,"POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
25,Ares,15004,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.112511,2041 - 2060,days,"MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
32,Arteixo,15005,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.188190,2041 - 2060,days,"MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


In [460]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z12 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Duracion_sequias/municipios.mbtiles -l Duracion_sequias -Z1 -z12 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Duracion_sequias/municipios.json


8115 features, 10702553 bytes of geometry, 52417 bytes of separate metadata, 329422 bytes of string pool


Task created
Finished processing


## Mapa de cultivos
**Read data**

In [19]:
gdf_corine = gpd.read_file('../../datasets/raw/crops/CLC2018_GDB/CLC2018_ES.gdb', driver="FileGDB", layer='CLC18_ES')
# Re-Project
gdf_corine = gdf_corine.to_crs("EPSG:4326")
gdf_corine = gdf_corine.set_crs("EPSG:4326")

**Reorganice data**

In [21]:
dic = {'Cereal': ['211', '212'],
'Olivar': ['223'],
'Dehesa': ['244'],
'Viñedo': ['221']}

gdf = gpd.GeoDataFrame(columns=['ID', 'value', 'geometry'])

for crop in dic.keys():
    print(crop)
    gdf_tmp = gdf_corine[gdf_corine['CODE_18'].isin(dic[crop])].copy()
    gdf_tmp['value'] = crop
    gdf_tmp = gdf_tmp[['ID', 'value', 'geometry']]
    
    gdf  = pd.concat([gdf, gdf_tmp])

gdf['dataset'] = 'Mapa de cultivos'
gdf['indicator'] = 'Localización cultivo'

gdf = gdf[['ID', 'dataset', 'indicator', 'value', 'geometry']]

gdf.head()

Cereal
Olivar
Dehesa
Viñedo


,ID,dataset,indicator,value,geometry
16379,16382,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.64246 36.12721, -5.64226 36..."
16380,16383,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.50050 36.17831, -5.50045 36..."
16381,16384,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.50964 36.18659, -5.50953 36..."
16382,16385,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.69228 36.18296, -5.69232 36..."
16383,16386,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.46238 36.20897, -5.46238 36..."


In [23]:
dic = {'Cereal': 2,
'Olivar': 1,
'Dehesa': 4,
'Viñedo': 3}
gdf['crop_name'] = gdf['value']
gdf['value'] = gdf['crop_name'].apply(lambda x: dic[x])
gdf

,ID,dataset,indicator,value,geometry,crop_name
16379,16382,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.64246 36.12721, -5.64226 36...",Cereal
16380,16383,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.50050 36.17831, -5.50045 36...",Cereal
16381,16384,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.50964 36.18659, -5.50953 36...",Cereal
16382,16385,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.69228 36.18296, -5.69232 36...",Cereal
16383,16386,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.46238 36.20897, -5.46238 36...",Cereal
...,...,...,...,...,...,...
48671,48674,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-6.66044 42.73515, -6.66040 42...",Viñedo
48672,48675,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-1.78326 42.79147, -1.78299 42...",Viñedo
48673,48676,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-7.62373 42.80460, -7.62367 42...",Viñedo
48674,48677,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-2.65395 43.22925, -2.65397 43...",Viñedo


In [25]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Mapa_cultivos'
name = 'Mapa_cultivos'
geo_name= 'Mapa_cultivos'

# Save GeoDataFrame as `GeoJSON`
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, folder_name, opts="-Z1 -z10 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Mapa_cultivos/Mapa_cultivos.mbtiles -l Mapa_cultivos -Z1 -z10 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Mapa_cultivos/Mapa_cultivos.json


44488 features, 116252109 bytes of geometry, 560 bytes of separate metadata, 316439 bytes of string pool
tile 5/15/12 size is 971178 with detail 12, >500000    
Going to try keeping the sparsest 46.34% of the features to make it fit
tile 5/15/12 size is 505878 with detail 12, >500000    
Going to try keeping the sparsest 41.22% of the features to make it fit
tile 6/31/24 size is 1263974 with detail 12, >500000    
Going to try keeping the sparsest 35.60% of the features to make it fit
tile 6/31/23 size is 627836 with detail 12, >500000    
Going to try keeping the sparsest 71.67% of the features to make it fit
tile 7/62/48 size is 660450 with detail 12, >500000    
Going to try keeping the sparsest 68.14% of the features to make it fit
tile 7/63/47 size is 525389 with detail 12, >500000    
Going to try keeping the sparsest 85.65% of the features to make it fit
tile 7/62/49 size is 687601 with detail 12, >500000    
Going to try keeping the sparsest 65.44% of the features to make it fi

Task created
Finished processing


#### **Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format**

In [352]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Mapa_cultivos'
name = 'Mapa_cultivos'

vector_tile_processing(path, name, folder_name, gdf, pbf_format=True, upload_to_GCS=True)

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Mapa_cultivos/municipios.mbtiles -l Mapa_cultivos -Z1 -z12 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Mapa_cultivos/municipios.json


44488 features, 116648162 bytes of geometry, 4838 bytes of separate metadata, 316416 bytes of string pool
tile 5/15/12 size is 965100 with detail 12, >500000    
Going to try keeping the sparsest 46.63% of the features to make it fit
tile 5/15/12 size is 510028 with detail 12, >500000    
Going to try keeping the sparsest 41.14% of the features to make it fit
tile 6/31/24 size is 1255515 with detail 12, >500000    
Going to try keeping the sparsest 35.84% of the features to make it fit
tile 6/31/23 size is 624548 with detail 12, >500000    
Going to try keeping the sparsest 72.05% of the features to make it fit
tile 7/63/47 size is 521119 with detail 12, >500000    
Going to try keeping the sparsest 86.35% of the features to make it fit
tile 7/62/48 size is 656341 with detail 12, >500000    
Going to try keeping the sparsest 68.56% of the features to make it fit
tile 7/62/49 size is 682412 with detail 12, >500000    
Going to try keeping the sparsest 65.94% of the features to make it f

Task created
Finished processing
Converting from MBTiles to pbf
Processing: mb-util ../../datasets/processed/MBTiles/Mapa_cultivos/municipios.mbtiles ../../datasets/processed/MBTiles/Mapa_cultivos/municipios --image_format=pbf


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:../../datasets/processed/MBTiles/Mapa_cultivos/municipios.mbtiles --> ../../datasets/processed/MBTiles/Mapa_cultivos/municipios
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 11354 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 

Task created
Finished processing
Uploading pbf files to GCS
File ../../datasets/processed/MBTiles/Mapa_cultivos/municipios/12/1962/1501.pbf uploaded to MBTiles/Mapa_cultivos/municipios/municipios/12/1962/1501.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/municipios/12/1962/1522.pbf uploaded to MBTiles/Mapa_cultivos/municipios/municipios/12/1962/1522.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/municipios/12/1962/1508.pbf uploaded to MBTiles/Mapa_cultivos/municipios/municipios/12/1962/1508.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/municipios/12/1962/1556.pbf uploaded to MBTiles/Mapa_cultivos/municipios/municipios/12/1962/1556.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/municipios/12/1962/1513.pbf uploaded to MBTiles/Mapa_cultivos/municipios/municipios/12/1962/1513.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/municipios/12/1962/1588.pbf uploaded to MBTiles/Mapa_cultivos/municipios/municipios/12/1962/1588.pbf.
File ../../datas

## Proyecciones de rendimiento del olivo.
**Create all tiles**

In [180]:
dataset = 'olive_grove_indicators'
indicator = 'yield_change_per'
path = '../../datasets/processed/MBTiles'
folder_name = 'Proyecciones_rendimiento'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Create `MBTile` for a single `scenario` and `year`**

In [369]:
dataset = 'olive_grove_indicators'
indicator = 'yield_change_per'
path = '../../datasets/processed/MBTiles'
folder_name = 'Proyecciones_rendimiento_olivar'
geo_type = 'Municipios'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2041–2070')]
gdf.head()

,NAMEUNIT,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
0,Abegondo,15001,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
1,Ames,15002,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
2,Aranga,15003,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
3,Ares,15004,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
4,Arteixo,15005,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


In [29]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z12 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Proyecciones_rendimiento/municipios.mbtiles -l Proyecciones_rendimiento -Z1 -z12 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Proyecciones_rendimiento/municipios.json


8115 features, 10691777 bytes of geometry, 49924 bytes of separate metadata, 203346 bytes of string pool


Task created
Finished processing


  99.9%  12/2022/1598  


## Cambios en zonas óptimas para la producción de vino de calidad.

In [514]:
dataset = 'vineyard_indicators'
indicator = 'zonas_alto_potencial_climático'
path = '../../datasets/processed/MBTiles'
folder_name = 'Zonas_alto_potencial_climático_viñedo'
geo_types = ['Rejilla']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

Rejilla
Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.mbtiles -l Zonas_alto_potencial_climático_viñedo -Z1 -z10 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.json


701 features, 176287 bytes of geometry, 32 bytes of separate metadata, 10460 bytes of string pool
  99.9%  10/504/388  


Task created
Finished processing
Converting from MBTiles to pbf
Processing: mb-util ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.mbtiles ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla --image_format=pbf


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.mbtiles --> ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 620 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 620 tiles exported
DEBUG:mbutil.util:flipping
IN

Task created
Finished processing
Uploading pbf files to GCS
File ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/2/2/1.pbf uploaded to MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/rejilla/2/2/1.pbf.
File ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/2/1/1.pbf uploaded to MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/rejilla/2/1/1.pbf.
File ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/4/7/6.pbf uploaded to MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/rejilla/4/7/6.pbf.
File ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/4/7/5.pbf uploaded to MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/rejilla/4/7/5.pbf.
File ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/4/8/6.pbf uploaded to MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla/rejilla/4/8/6.pbf.
File ../../datasets/processed/MBTiles/Zonas_a

**Create `MBTile` for a single `scenario` and `year`**

In [522]:
dataset = 'vineyard_indicators'
indicator = 'zonas_alto_potencial_climático'
path = '../../datasets/processed/MBTiles'
folder_name = 'Zonas_alto_potencial_climático_viñedo'
geo_type = 'Rejilla'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2021-2050')]
gdf.head()

,grid_id,dataset,indicator,scenario,value,year,unit,geometry
1402,62,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,NaN,2021-2050,NaN,"POLYGON ((-3.90000 37.10000, -3.70000 37.10000..."
1403,63,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,NaN,2021-2050,NaN,"POLYGON ((-3.70000 37.10000, -3.50000 37.10000..."
1404,64,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,7.0,2021-2050,NaN,"POLYGON ((-3.50000 37.10000, -3.30000 37.10000..."
1405,65,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,7.0,2021-2050,NaN,"POLYGON ((-3.30000 37.10000, -3.10000 37.10000..."
1406,66,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,12.0,2021-2050,NaN,"POLYGON ((-3.10000 37.10000, -2.90000 37.10000..."


In [523]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
with open(f"{path}/{folder_name}/{geo_name}.json", 'w') as outfile:
    outfile.write(gdf.to_json())

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z10 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.mbtiles -l Zonas_alto_potencial_climático_viñedo -Z1 -z10 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.json


701 features, 44875 bytes of geometry, 32 bytes of separate metadata, 4012 bytes of string pool


Task created
Finished processing


  99.9%  10/497/381  
